# ML pipeline: From Raw Data to Predictions

Reference: DataCamp Data Scientist with Python Course<br>
Data Source: Data Driven 

In [49]:
from google.colab import files
files.upload()

{}

In [0]:
%matplotlib inline
from __future__ import division
from __future__ import print_function

# ignore deprecation warnings in sklearn
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import os
import sys

# add the 'src' directory as one where we can import modules
src_dir = os.path.join(os.getcwd(), os.pardir, 'src')
sys.path.append(src_dir)

from multilabel import multilabel_sample_dataframe, multilabel_train_test_split
from SparseInteractions import SparseInteractions
from metrics import multi_multi_log_loss

In [0]:
from sklearn.feature_selection import chi2, SelectKBest

from sklearn.pipeline import Pipeline, FeatureUnion

from sklearn.impute import SimpleImputer
from sklearn.feature_extraction.text import HashingVectorizer

from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import MaxAbsScaler

from sklearn.feature_extraction.text import CountVectorizer

# Load Data
Data for this notebook has been collected as below:
 - [Sign up for an account on DrivenData](http://www.drivendata.org)
 - [Join the Box-plots for education competition](https://www.drivendata.org/competitions/46/box-plots-for-education-reboot/)
 
 

In [27]:
path_to_training_data='drive/My Drive/TrainingData.csv'
df = pd.read_csv(path_to_training_data, index_col=0)

print(df.shape)

(400277, 25)


# Resample Data

Dataset shape is too big, for computational efficiency I have resampled and also created dummy variables for our labels and split our sampled dataset into a training set and a test set.

In [0]:
LABELS = ['Function',
          'Use',
          'Sharing',
          'Reporting',
          'Student_Type',
          'Position_Type',
          'Object_Type', 
          'Pre_K',
          'Operating_Status']

NON_LABELS = [c for c in df.columns if c not in LABELS]

SAMPLE_SIZE = 40000

sampling = multilabel_sample_dataframe(df,
                                       pd.get_dummies(df[LABELS]),
                                       size=SAMPLE_SIZE,
                                       min_count=25,
                                       seed=43)

dummy_labels = pd.get_dummies(sampling[LABELS])

X_train, X_test, y_train, y_test = multilabel_train_test_split(sampling[NON_LABELS],
                                                               dummy_labels,
                                                               0.2,
                                                               min_count=3,
                                                               seed=43)

In [0]:
from sklearn.metrics.scorer import make_scorer

log_loss_scorer = make_scorer(multi_multi_log_loss)

# Create preprocessing tools

We need tools to preprocess our text and numeric data. We'll create those tools here. The `combine_text_columns` function will take a DataFrame of text columns and return a single series where all of the text in the columns has been joined together.

We'll then create `FunctionTransformer` objects that select our text and numeric data from the dataframe.

Finally, we create a custom scoring method that uses the `multi_multi_log_loss` function that is the evaluation metric for the competition.

In [0]:
NUMERIC_COLUMNS = ['FTE', "Total"]

def combine_text_columns(data_frame, to_drop=NUMERIC_COLUMNS + LABELS):
    """ Takes the dataset as read in, drops the non-feature, non-text columns and
        then combines all of the text columns into a single vector that has all of
        the text for a row.
        
        :param data_frame: The data as read in with read_csv (no preprocessing necessary)
        :param to_drop (optional): Removes the numeric and label columns by default.
    """
    # drop non-text columns that are in the df
    to_drop = set(to_drop) & set(data_frame.columns.tolist())
    text_data = data_frame.drop(to_drop, axis=1)
    
    # replace nans with blanks
    text_data.fillna("", inplace=True)
    
    # joins all of the text items in a row (axis=1)
    # with a space in between
    return text_data.apply(lambda x: " ".join(x), axis=1)


In [0]:
from sklearn.preprocessing import FunctionTransformer

get_text_data = FunctionTransformer(combine_text_columns, validate=False)
get_numeric_data = FunctionTransformer(lambda x: x[NUMERIC_COLUMNS], validate=False)

In [32]:
get_text_data.fit_transform(sampling.head(10))

38      OTHER PURCHASED SERVICES  SCHOOL-WIDE SCHOOL P...
70      Extra Duty Pay/Overtime For Support Personnel ...
198     Supplemental *  Operation and Maintenance of P...
209     REPAIR AND MAINTENANCE SERVICES  PUPIL TRANSPO...
614      GENERAL EDUCATION LOCAL EDUCATIONAL AIDE,70 H...
662     EMPLOYEE BENEFITS EARLY EDUCATION FEDERAL GDPG...
750     Personal Services - Teachers   TCHER 5TH GRADE...
931     General Supplies      General Supplies  Instru...
1265    UNEMPLOYMENT COMPENSATION            School  B...
1350    TRANSPORTATION/FIELD TRIPS  GENERAL FUND (blan...
dtype: object

In [33]:
get_numeric_data.fit_transform(sampling.head(10))

,FTE,Total
38,NaN,653.460000
70,NaN,2153.530000
198,NaN,-8291.860000
209,NaN,618.290000
614,0.71,21747.666875
662,NaN,0.630000
750,1.00,49768.820000
931,NaN,-1.020000
1265,NaN,8574.980000
1350,NaN,2740.850000


# Train model pipeline

Does the necessary preprocessing such as mean imputation, scaling, and trains the classifier.

In [47]:
TOKENS_ALPHANUMERIC = '[A-Za-z0-9]+(?=\\s+)'
%timeit

# set a reasonable number of features before adding interactions
chi_k = 300

# create the pipeline object
pl = Pipeline([
        ('union', FeatureUnion(
            transformer_list = [
                ('numeric_features', Pipeline([
                    ('selector', get_numeric_data),
                    ('imputer', SimpleImputer())
                ])),
                ('text_features', Pipeline([
                    ('selector', get_text_data),
                    ('vectorizer', CountVectorizer()),
                    ('dim_red', SelectKBest(chi2, chi_k))
                ]))
             ]
        )),
        ('int', SparseInteractions(degree=2)),
        ('scale', MaxAbsScaler()),
        ('clf', OneVsRestClassifier(LogisticRegression()))
    ])

# fit the pipeline to our training data
pl.fit(X_train, y_train.values)

# print the score of our trained pipeline on our test set
print("Logloss score of trained pipeline: ", log_loss_scorer(pl, X_test, y_test.values))

Logloss score of trained pipeline:  1.6709244720155212


# Predict holdout set and write to file

Finally, we want to use our trained pipeline to predict the holdout dataset. 

In [0]:
path_to_holdout_data = 'drive/My Drive/TestData.csv'

# Load holdout data
holdout = pd.read_csv(path_to_holdout_data, index_col=0)

# Make predictions
predictions = pl.predict_proba(holdout)

# Format correctly in new DataFrame: prediction_df
prediction_df = pd.DataFrame(columns=pd.get_dummies(df[LABELS]).columns,
                             index=holdout.index,
                             data=predictions)


# Save prediction_df to csv called "predictions.csv"
prediction_df.to_csv("predictions.csv")